In [0]:
import numpy as np
import pandas as pd
import copy

#!pip install urllib
import urllib.request as url 

#!pip install bs4
from bs4 import BeautifulSoup as bs

#!pip install wordcloud
from wordcloud import WordCloud

import re
import requests
import matplotlib.pyplot as plt
%matplotlib inline

import random
random.seed(123)

import spacy
nlp = spacy.load("en_core_web_sm")

import nltk
import random

from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import word_tokenize

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [0]:
links10K = pd.read_csv("10k_filing_info_train_links-1570100710859.csv", sep= ',')

In [0]:
data = {'ticker':[], 'cik':[], '10k_link':[], 'text':[]}

In [0]:
for rownum, link10k in links10K.iterrows():
   print(rownum)
   ticker = link10k['ticker']
   cik = link10k['cik']
   link = link10k['10k_link']
   try:
    txt = url.urlopen(link).read()
    soup = bs(txt, 'html.parser')
    for type_tag in soup.find_all('type'):
      if type_tag.text[0:4] == '10-K':
        data['text'].append(type_tag.html.text)
        data['ticker'].append(ticker)
        data['cik'].append(cik)
        data['10k_link'].append(link)
        break
   except:
    print(rownum)
    pass

0
1
2
3
3
4
5
6
6
7
7
8
8
9
9
10
10
11
11
12
13
13
14
15
16
17
18
19
19
20
20
21
22
23
24
25
25
26
26
27
28
29
30
31
31
32
33
33
34
35
35
36
37
38
38
39
40
41
42
43
44
45
45
46
46
47
48
49
49
50
51
52
52
53
53
54
54
55
56
56
57
58
59
60
61
62
63
64
64
65
65
66
66
67
67
68
69
70
70
71
72
73
73
74
75
76
77
78
78
79
80
81
82
83
83
84
85
86
87
88
89
89
90
91
91
92
93
93
94
94
95
96
96
97
98
99
100
101
102
102
103
103
104
104
105
105
106
107
108
109
110
111
112
112
113
114
115
115
116
117
118
119
120
121
122
123
124
125
126
127
127
128
129
130
131
132
132
133
133
134
135
136
137
138
139
140
141
142
142
143
144
145
146
147
147
148
149
149
150
150
151
152
152
153
154
155
156
157
158
159
160
161
162
163
164
164
165
166
167
168
169
170
171
172
173
173
174
174
175
176
177
178
179
180
180
181
182
183
184
184
185
185
186
187
187
188
188
189
190
191
191
192
193
194
195
195
196
196
197
198
199
200
201
201
202
203
204
204
205
206
207
208
209
209
210
210
211
212
213
214
215
215
216
216
217
218
218
219

In [0]:
pd.DataFrame(data).to_csv("Extracted_10Kdata.csv",sep=',', columns=['text', 'ticker', 'cik', '10k_link'], header=True, index=False)

In [0]:
data10k = pd.read_csv("Extracted_10Kdata.csv", sep= ',')

In [0]:
dictnry = pd.read_csv("finance_word_dictionary-1570171303593.csv", sep= ',')

In [0]:
positiveWords = dictnry.query('Positive == 1')['Word']

In [0]:
negativeWords = dictnry.query('Negative == 1')['Word']

In [0]:
dictWords = pd.concat([positiveWords, negativeWords])

In [0]:
dictWords.values

array(['ABLE', 'ABUNDANCE', 'ABUNDANT', ..., 'WRONGFUL', 'WRONGFULLY',
       'WRONGLY'], dtype=object)

In [0]:
data10k['text'] = [text[:1000000] for text in data10k['text']]
data10k['text_length'] = data10k['text'].apply(len)
data10k.head()

,text,ticker,cik,10k_link,text_length
0,\n\n\n\nFE-12.31.2011-10K\n\n\n UNITED STATES ...,FE,1031296,https://www.sec.gov/Archives/edgar/data/103129...,1000000
1,\n\n\n\nCL-12.31.2014-10K \n\n\nUNITED STATES ...,CL,21665,https://www.sec.gov/Archives/edgar/data/21665/...,324353
2,\n\n\n\n10-K\n\n\nTable of Contents UNITED STA...,PRU,1137774,https://www.sec.gov/Archives/edgar/data/113777...,1000000
3,\n\ne10vk\n\n\n\nTable of Contents\n\n \n \nSE...,CAM,941548,https://www.sec.gov/Archives/edgar/data/941548...,98895
4,\n\n\n\nPNR-2014.12.31-10K\n\n\n UNITED STATES...,PNR,77360,https://www.sec.gov/Archives/edgar/data/77360/...,391778


In [0]:
def words_and_toLower(raw_review):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 
    words = letters_only.lower().split()
    return( " ".join(words))

In [0]:
data10k['text'] = [words_and_toLower(text) for text in data10k['text']]
data10k['text_length'] = data10k['text'].apply(len)
data10k.head()

,text,ticker,cik,10k_link,text_length
0,fe k united states securities and exchange com...,FE,1031296,https://www.sec.gov/Archives/edgar/data/103129...,895470
1,cl k united states securities and exchange com...,CL,21665,https://www.sec.gov/Archives/edgar/data/21665/...,291634
2,k table of contents united statessecurities an...,PRU,1137774,https://www.sec.gov/Archives/edgar/data/113777...,930398
3,e vk table of contents securities and exchange...,CAM,941548,https://www.sec.gov/Archives/edgar/data/941548...,85363
4,pnr k united states securities and exchange co...,PNR,77360,https://www.sec.gov/Archives/edgar/data/77360/...,355873


In [0]:
def postiveCount_and_negativeCount(doc, remove_stopwords = True):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    if remove_stopwords:
        tokens = [word.text for word in doc if not word.is_stop]
    else:
        tokens = [word.text for word in doc]

    filtered_tokens = []
    pos = 0
    neg = 0
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
      if re.search('[a-zA-Z]', token):
        if token.upper() in positiveWords.values:
          pos += 1
        if token.upper() in negativeWords.values:
          neg += 1
        filtered_tokens.append(token)    
    pol = (neg-pos)  
    return pol

def postiveWords_and_negativeWords_lemmatize(doc):
    tokens = [word for word in doc]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
      if token.text.upper() in dictWords.values:
        filtered_tokens.append(token)    
    lemma = [t.lemma_ for t in filtered_tokens]   
    return lemma  

In [0]:
data10k['pol'] = data10k['text'].apply(lambda x: postiveCount_and_negativeCount(nlp(x)))

In [0]:
op = {'id':[], 'ol':[]}

In [0]:
for i in range(855):
  op['id'].append(i)
  if(df_merge_col['id'] == i):
    op['ol'].append(df_merge_col.loc[df_merge_col['id'] == i]['id'].values[0])
  else
    op['ol'].append(1)

In [0]:
data10k['text'] = data10k['text'].apply(lambda x: postiveWords_and_negativeWords_lemmatize(nlp(x)))

In [0]:
pd.DataFrame(data10k).to_csv("Extracted_10Kdata1.csv",sep=',', columns=['text', 'long_term_outlook', 'ticker', 'cik', '10k_link'], header=True, index=False)

In [0]:
pd.DataFrame(op).to_csv("output.csv",sep=',', columns=['id', 'ol'], header=True, index=False)

In [0]:
def words_to_string(words):    
    return( " ".join(words))

In [0]:
allData = []
for index, row in data10k.iterrows():
  allData.append(words_to_string(row['text']))

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(stop_words='english',lowercase=True, decode_error='ignore')

tdm = cv.fit_transform(allData)
type(tdm)

In [0]:
Mat = tdm.todense()
print(Mat.shape)
Mat.head()

In [0]:
Mat['type'] = data10k['long_term_outlook']
Mat.head()

In [0]:
Mat = Mat.sample(frac = 1,random_state=123)
train = Mat.iloc[:1500]
test = Mat.iloc[1500:]

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
logreg.fit(X_train,y_train)

In [0]:
y_pred=logreg.predict(X_train)

In [0]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

In [0]:
from sklearn.metrics import classification_report

print("Classification Report on Train Data")
print(classification_report(y_train,y_pred1,digits=2))
print("\n")

print("Classification Report on Test Data")
print(classification_report(y_test,y_pred,digits=2))

In [0]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
# classifier.fit(train.iloc[:,:-1],train.iloc[:,-1])

classifier.fit(X_train,y_train)

# Predictions on train data
y_pred1=classifier.predict(X_train)
print(confusion_matrix(y_train,y_pred1))

# Predictions on test data
y_pred2=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred2))

In [0]:
from sklearn.metrics import classification_report

print("Classification Report on Train Data")
print(classification_report(y_train,y_pred1,digits=2))
print("\n")

print("Classification Report on Test Data")
print(classification_report(y_test,y_pred2,digits=2))

In [0]:
data10k['text'] = [' '.join(text) for text in data10k['text']]

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=20000,
                                 min_df=0.2,
                                 use_idf=True, ngram_range=(1,4))

tfidf_matrix = tfidf_vectorizer.fit_transform(data10k['text'])

print(tfidf_matrix.shape)

In [0]:
terms = tfidf_vectorizer.get_feature_names()
print(type(terms))
terms[:5]

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
import joblib
Sum_of_squared_distances = []
kmeanModel = KMeans(n_clusters=2, random_state=143)
kmeanModel.fit(tfidf_matrix)
Sum_of_squared_distances.append(kmeanModel.inertia_)

In [0]:
num_clusters = 2

km = KMeans(n_clusters=num_clusters)

km.fit(tfidf_matrix)
clusters = km.labels_.tolist()
centers = km.cluster_centers_
print(f"the cluster centers are {centers}")

joblib.dump(km,  'doc_cluster_best_K.pkl')

In [0]:
print(km.cluster_centers_)
print(km.cluster_centers_.shape)

In [0]:
km.cluster_centers_.argsort()

In [0]:
## Reversing the list so that index of max element is in 0th index
km.cluster_centers_.argsort()[:,::-1]

In [0]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
for i in range(num_clusters):
    print("\n")
    print("Top words in Cluster-%d :" % i, end='')
    print("\n")
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print('%s' % terms[ind].split(' '), end=',')